# 주식 가격 변동 모니터링 및 뉴스 분석 프로그램

이 노트북은 주식의 가격 변동을 모니터링하고, 큰 변동이 있을 때 관련 뉴스를 자동으로 검색하는 프로그램을 설명합니다.

## 목차
1. 프로그램 개요
2. 필요한 라이브러리 설치
3. API 설정
4. 코드 구조 상세 설명
5. 실행 예제

## 1. 프로그램 개요

### 주요 기능:
- **주식 가격 조회**: Alpha Vantage API를 사용해 실시간 주가 데이터를 가져옵니다
- **가격 변동률 계산**: 어제와 오늘의 종가를 비교해 변동률을 계산합니다
- **뉴스 검색**: 1% 이상 변동 시 NewsAPI를 통해 관련 뉴스를 자동으로 검색합니다
- **결과 출력**: 주가 정보와 뉴스를 보기 좋게 포맷팅해서 출력합니다

### 사용하는 API:
1. **Alpha Vantage**: 무료 주식 데이터 API
2. **NewsAPI**: 뉴스 검색 API

## 2. 필요한 라이브러리 설치

프로그램 실행에 필요한 라이브러리를 설치합니다:

In [ ]:

# 터미널에 설치 필요
!uv add --dev jupyter ipykernel

: 

In [24]:
# requests 라이브러리 설치 (HTTP 요청을 위해 필요)
!uv add requests

Resolved 104 packages in 8ms
Audited 5 packages in 1ms


## 3. API 설정

### Alpha Vantage API 키 얻기:
1. https://www.alphavantage.co/support/#api-key 방문
2. 무료 API 키 신청 (이메일 입력)
3. 받은 API 키를 코드에 입력

### NewsAPI 키 얻기:
1. https://newsapi.org/register 방문
2. 무료 계정 생성
3. API 키를 코드에 입력

## 4. 코드 구조 상세 설명

### 4.1 필요한 라이브러리 임포트

In [3]:
import requests  # HTTP 요청을 보내기 위한 라이브러리
from typing import Dict, List, Tuple, Optional  # 타입 힌트를 위한 라이브러리

### 4.2 StockAnalyzer 클래스 정의

클래스를 사용하여 관련 기능들을 하나로 묶어 관리합니다:

In [14]:
class StockAnalyzer:
    """주식 데이터 분석 및 뉴스 모니터링 클래스"""
    
    def __init__(self, ticker: str, company: str):
        """
        초기화 메서드
        
        매개변수:
        - ticker: 주식 심볼 (예: 'TSLA', 'AAPL')
        - company: 회사명 (예: 'Tesla Inc', 'Apple Inc')
        """
        self.ticker = ticker  # 주식 심볼 저장
        self.company = company  # 회사명 저장
        
        # API 엔드포인트 URL 저장
        self.api_endpoints = {
            'stock': "https://www.alphavantage.co/query",  # 주식 데이터 API
            'news': "https://newsapi.org/v2/everything"     # 뉴스 API
        }
        
        # API 키 저장 (실제 사용 시 자신의 키로 교체 필요)
        self.credentials = {
            'alpha_vantage': "EOWMRQVFKZNV4IFL",
            'news_api': "008196f15126437588a9e0e1087f2652"
        }

### 4.3 주식 데이터 가져오기 메서드

In [15]:
def fetch_stock_data(self) -> Dict:
    """
    Alpha Vantage API를 통해 주식 데이터 조회
    
    반환값:
    - 주식 데이터가 담긴 딕셔너리 (JSON 형식)
    - 오류 발생 시 빈 딕셔너리 반환
    """
    # API 요청에 필요한 매개변수 설정
    parameters = {
        "function": "TIME_SERIES_DAILY",  # 일별 시계열 데이터 요청
        "symbol": self.ticker,            # 주식 심볼
        "apikey": self.credentials['alpha_vantage']  # API 키
    }
    
    try:
        # API에 GET 요청 보내기
        response = requests.get(self.api_endpoints['stock'], params=parameters)
        response.raise_for_status()  # HTTP 오류 확인
        return response.json()       # JSON 형식으로 반환
    except requests.exceptions.RequestException as e:
        print(f"주식 데이터 조회 실패: {e}")
        return {}

### 4.4 종가 추출 메서드

In [16]:
def extract_closing_prices(self, stock_data: Dict) -> Tuple[Optional[float], Optional[float]]:
    """
    최근 이틀간의 종가 추출
    
    매개변수:
    - stock_data: API에서 받은 주식 데이터
    
    반환값:
    - (오늘 종가, 어제 종가) 튜플
    - 데이터가 없으면 (None, None) 반환
    """
    try:
        # "Time Series (Daily)" 키에서 일별 데이터 추출
        daily_series = stock_data.get("Time Series (Daily)", {})
        
        # 날짜를 최신순으로 정렬
        dates = sorted(daily_series.keys(), reverse=True)
        
        # 최소 2일치 데이터가 필요
        if len(dates) < 2:
            return None, None
        
        # 가장 최근 날짜의 종가 ("4. close"는 종가를 의미)
        recent_close = float(daily_series[dates[0]]["4. close"])
        
        # 그 전날의 종가
        previous_close = float(daily_series[dates[1]]["4. close"])
        
        return recent_close, previous_close
        
    except (KeyError, ValueError, IndexError):
        # 데이터 형식이 잘못되었거나 필요한 키가 없는 경우
        return None, None

### 4.5 가격 변동률 계산 메서드

In [17]:
def calculate_price_change(self, current: float, previous: float) -> Tuple[float, float, str]:
    """
    가격 변동률 계산
    
    매개변수:
    - current: 현재 가격
    - previous: 이전 가격
    
    반환값:
    - (변동 금액, 변동 퍼센트, 상승/하락 이모지)
    """
    # 변동 금액 계산
    change_amount = current - previous
    
    # 변동률 계산 (백분율)
    change_percent = (change_amount / previous) * 100
    
    # 상승/하락 표시 이모지 선택
    indicator = "📈" if change_amount >= 0 else "📉"
    
    return change_amount, round(change_percent), indicator

### 4.6 뉴스 검색 메서드

In [18]:
def fetch_company_news(self, min_threshold: float = 1.0) -> List[Dict]:
    """
    뉴스 API를 통해 관련 뉴스 조회
    
    매개변수:
    - min_threshold: 뉴스를 검색할 최소 변동률 (기본값 1%)
    
    반환값:
    - 뉴스 기사 리스트 (최대 3개)
    """
    # NewsAPI 요청 매개변수
    params = {
        "apiKey": self.credentials['news_api'],  # API 키
        "qInTitle": self.company,                # 제목에 회사명 포함
        "sortBy": "publishedAt",                 # 최신순 정렬
        "pageSize": 3                            # 최대 3개 기사
    }
    
    try:
        # API 요청
        response = requests.get(self.api_endpoints['news'], params=params)
        response.raise_for_status()
        news_data = response.json()
        
        # "articles" 키에서 기사 목록 추출 (최대 3개)
        return news_data.get("articles", [])[:3]
    except requests.exceptions.RequestException as e:
        print(f"뉴스 조회 실패: {e}")
        return []

### 4.7 뉴스 포맷팅 메서드

In [19]:
def format_news_item(self, article: Dict, price_info: Tuple[str, float, str]) -> str:
    """
    뉴스 항목을 보기 좋게 포맷팅
    
    매개변수:
    - article: 뉴스 기사 데이터
    - price_info: (주식 심볼, 변동률, 상승/하락 표시)
    
    반환값:
    - 포맷팅된 뉴스 문자열
    """
    ticker, percent, indicator = price_info
    
    # 뉴스 포맷 생성
    return (f"[{ticker}] {indicator} {abs(percent)}% 변동\n"
            f"제목: {article.get('title', '제목 없음')}\n"
            f"요약: {article.get('description', '내용 없음')[:200]}...")  # 요약은 200자로 제한

### 4.8 메인 분석 메서드

In [20]:
def analyze_and_report(self):
    """
    주식 분석 및 리포트 생성 - 전체 프로세스를 실행하는 메인 메서드
    """
    # 1단계: 주식 데이터 가져오기
    print("주식 데이터를 가져오는 중...")
    stock_data = self.fetch_stock_data()
    if not stock_data:
        print("주식 데이터를 가져올 수 없습니다.")
        return
    
    # 2단계: 종가 추출
    today_close, yesterday_close = self.extract_closing_prices(stock_data)
    if today_close is None or yesterday_close is None:
        print("종가 데이터를 추출할 수 없습니다.")
        return
    
    # 3단계: 가격 정보 출력
    print(f"\n[{self.ticker}] 주가 정보")
    print(f"최근 종가: ${today_close:.2f}")
    print(f"이전 종가: ${yesterday_close:.2f}")
    
    # 4단계: 변동률 계산
    change, percent_change, trend = self.calculate_price_change(today_close, yesterday_close)
    print(f"변동률: {percent_change}% ({trend})")
    
    # 5단계: 뉴스 확인 (1% 이상 변동 시)
    if abs(percent_change) >= 1:
        print("\n📰 관련 뉴스를 검색합니다...")
        articles = self.fetch_company_news()
        
        if articles:
            print(f"\n최신 {len(articles)}개 뉴스:\n")
            for idx, article in enumerate(articles, 1):
                news_text = self.format_news_item(
                    article, 
                    (self.ticker, percent_change, trend)
                )
                print(f"[뉴스 {idx}]")
                print(news_text)
                print("-" * 60)
        else:
            print("관련 뉴스를 찾을 수 없습니다.")
    else:
        print("\n변동률이 1% 미만이므로 뉴스를 검색하지 않습니다.")

## 5. 실행 예제

### 5.1 전체 코드 실행

In [22]:
# 전체 코드를 한 번에 실행합니다
import requests
from typing import Dict, List, Tuple, Optional

class StockAnalyzer:
    """주식 데이터 분석 및 뉴스 모니터링 클래스"""
    
    def __init__(self, ticker: str, company: str):
        self.ticker = ticker
        self.company = company
        self.api_endpoints = {
            'stock': "https://www.alphavantage.co/query",
            'news': "https://newsapi.org/v2/everything"
        }
        self.credentials = {
            'alpha_vantage': "EOWMRQVFKZNV4IFL",
            'news_api': "008196f15126437588a9e0e1087f2652"
        }
        
    def fetch_stock_data(self) -> Dict:
        """Alpha Vantage API를 통해 주식 데이터 조회"""
        parameters = {
            "function": "TIME_SERIES_DAILY",
            "symbol": self.ticker,
            "apikey": self.credentials['alpha_vantage']
        }
        
        try:
            response = requests.get(self.api_endpoints['stock'], params=parameters)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"주식 데이터 조회 실패: {e}")
            return {}
    
    def extract_closing_prices(self, stock_data: Dict) -> Tuple[Optional[float], Optional[float]]:
        """최근 이틀간의 종가 추출"""
        try:
            daily_series = stock_data.get("Time Series (Daily)", {})
            dates = sorted(daily_series.keys(), reverse=True)
            
            if len(dates) < 2:
                return None, None
                
            recent_close = float(daily_series[dates[0]]["4. close"])
            previous_close = float(daily_series[dates[1]]["4. close"])
            
            return recent_close, previous_close
            
        except (KeyError, ValueError, IndexError):
            return None, None
    
    def calculate_price_change(self, current: float, previous: float) -> Tuple[float, float, str]:
        """가격 변동률 계산"""
        change_amount = current - previous
        change_percent = (change_amount / previous) * 100
        
        indicator = "📈" if change_amount >= 0 else "📉"
        
        return change_amount, round(change_percent), indicator
    
    def fetch_company_news(self, min_threshold: float = 1.0) -> List[Dict]:
        """뉴스 API를 통해 관련 뉴스 조회"""
        params = {
            "apiKey": self.credentials['news_api'],
            "qInTitle": self.company,
            "sortBy": "publishedAt",
            "pageSize": 3
        }
        
        try:
            response = requests.get(self.api_endpoints['news'], params=params)
            response.raise_for_status()
            news_data = response.json()
            return news_data.get("articles", [])[:3]
        except requests.exceptions.RequestException as e:
            print(f"뉴스 조회 실패: {e}")
            return []
    
    def format_news_item(self, article: Dict, price_info: Tuple[str, float, str]) -> str:
        """뉴스 항목 포맷팅"""
        ticker, percent, indicator = price_info
        
        return (f"[{ticker}] {indicator} {abs(percent)}% 변동\n"
                f"제목: {article.get('title', '제목 없음')}\n"
                f"요약: {article.get('description', '내용 없음')[:200]}...")
    
    def analyze_and_report(self):
        """주식 분석 및 리포트 생성"""
        # 주식 데이터 가져오기
        stock_data = self.fetch_stock_data()
        if not stock_data:
            print("주식 데이터를 가져올 수 없습니다.")
            return
        
        # 종가 추출
        today_close, yesterday_close = self.extract_closing_prices(stock_data)
        if today_close is None or yesterday_close is None:
            print("종가 데이터를 추출할 수 없습니다.")
            return
        
        # 가격 정보 출력
        print(f"최근 종가: ${today_close:.2f}")
        print(f"이전 종가: ${yesterday_close:.2f}")
        
        # 변동률 계산
        change, percent_change, trend = self.calculate_price_change(today_close, yesterday_close)
        print(f"변동률: {percent_change}% ({trend})")
        
        # 뉴스 확인 (1% 이상 변동 시)
        if abs(percent_change) >= 1:
            print("\n📰 관련 뉴스를 검색합니다...")
            articles = self.fetch_company_news()
            
            if articles:
                print(f"\n최신 {len(articles)}개 뉴스:\n")
                for idx, article in enumerate(articles, 1):
                    news_text = self.format_news_item(
                        article, 
                        (self.ticker, percent_change, trend)
                    )
                    print(f"[뉴스 {idx}]")
                    print(news_text)
                    print("-" * 60)
            else:
                print("관련 뉴스를 찾을 수 없습니다.")
        else:
            print("\n변동률이 1% 미만이므로 뉴스를 검색하지 않습니다.")


def main():
    """메인 실행 함수"""
    # Tesla 주식 모니터링 객체 생성
    monitor = StockAnalyzer(ticker="TSLA", company="Tesla Inc")
    
    # 분석 실행
    monitor.analyze_and_report()

# 실행
if __name__ == "__main__":
    main()

최근 종가: $321.20
이전 종가: $325.59
변동률: -1% (📉)

📰 관련 뉴스를 검색합니다...

최신 3개 뉴스:

[뉴스 1]
[TSLA] 📉 1% 변동
제목: INVESTOR ALERT: Pomerantz Law Firm Investigates Claims On Behalf of Investors of Tesla, Inc. - TSLA
요약: NEW YORK, July 28, 2025 (GLOBE NEWSWIRE) -- Pomerantz LLP is investigating claims on behalf of investors of  Tesla, Inc. (“Tesla” or the “Company”) (NASDAQ: TSLA). Such investors are advised to contac...
------------------------------------------------------------
[뉴스 2]
[TSLA] 📉 1% 변동
제목: Tesla, Inc. (TSLA)’s “A Cult Stock,” Says Jim Cramer
요약: We recently published 14 Stocks Jim Cramer Discussed As He Went “All In” On Quantum Computing. Tesla, Inc. (NASDAQ:TSLA) is one of the stocks Jim Cramer......
------------------------------------------------------------
[뉴스 3]
[TSLA] 📉 1% 변동
제목: Tesla, Inc. (TSLA): A Bull Case Theory
요약: We came across a bullish thesis on Tesla, Inc. on Stock Analysis Compilation’s Substack. In this article, we will summarize the bulls’ thesis on TSLA. Tesla,..

### 5.2 다른 주식 모니터링하기

다른 회사의 주식을 모니터링하려면 ticker와 company를 변경하면 됩니다:

In [23]:
# Apple 주식 모니터링
apple_monitor = StockAnalyzer(ticker="AAPL", company="Apple Inc")
apple_monitor.analyze_and_report()

# Microsoft 주식 모니터링
ms_monitor = StockAnalyzer(ticker="MSFT", company="Microsoft")
ms_monitor.analyze_and_report()

최근 종가: $211.27
이전 종가: $214.05
변동률: -1% (📉)

📰 관련 뉴스를 검색합니다...

최신 3개 뉴스:

[뉴스 1]
[AAPL] 📉 1% 변동
제목: Apple Inc. Investors: Please contact the Portnoy Law Firm to recover your losses. August 19, 2025 Deadline to file Lead Plaintiff Motion.
요약: Investors can contact the law firm at no cost to learn more about recovering their losses  LOS ANGELES, July  28, 2025  (GLOBE NEWSWIRE) -- The Portnoy......
------------------------------------------------------------
[뉴스 2]
[AAPL] 📉 1% 변동
제목: AAPL LAWSUIT: Apple Inc. Sued for Fraud over AI Promises to Investors – Contact BFA Law by August 19 Court Deadline (NASDAQ: AAPL)
요약: Apple Inc. investors that lost money are notified to contact BFA Law before the August 19, 2025 securities fraud class action deadline..........
------------------------------------------------------------
[뉴스 3]
[AAPL] 📉 1% 변동
제목: SHAREHOLDER ALERT Bernstein Liebhard LLP Announces A Securities Fraud Class Action Lawsuit Has Been Filed Against Apple Inc. (NASDAQ: AAPL)
요약: SH

## 6. 프로그램 흐름도

```
시작
 ↓
StockAnalyzer 객체 생성 (주식 심볼, 회사명 입력)
 ↓
Alpha Vantage API에서 주식 데이터 가져오기
 ↓
최근 2일간의 종가 추출
 ↓
가격 변동률 계산
 ↓
변동률이 1% 이상인가?
 ├─ 예 → NewsAPI에서 관련 뉴스 검색 → 뉴스 출력
 └─ 아니오 → "뉴스 검색 안 함" 메시지
 ↓
종료
```

## 7. 자주 발생하는 오류와 해결방법

### 7.1 API 키 오류
- **증상**: "Invalid API key" 메시지
- **해결**: API 키가 올바른지 확인하고, 활성화되었는지 확인

### 7.2 네트워크 오류
- **증상**: "Connection Error" 메시지
- **해결**: 인터넷 연결 확인, 방화벽 설정 확인

### 7.3 API 호출 제한
- **증상**: "Rate limit exceeded" 메시지
- **해결**: 
  - Alpha Vantage: 분당 5회, 일일 500회 제한
  - NewsAPI: 일일 100회 제한 (무료 계정)
  - 호출 간격을 늘리거나 유료 플랜 고려

## 8. 확장 아이디어

### 8.1 기능 추가
- 여러 주식 동시 모니터링
- 변동률 알림 임계값 사용자 설정
- 과거 데이터 분석 및 차트 생성
- 이메일/SMS 알림 기능

### 8.2 데이터 저장
- CSV/Excel 파일로 데이터 저장
- 데이터베이스 연동
- 일별/주별/월별 리포트 생성

### 8.3 UI 개선
- 웹 대시보드 만들기
- 실시간 업데이트 기능
- 모바일 앱 개발